In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
